In [1]:
import dash
from dash.dependencies import Input, Output
import dash_html_components as dhc
import dash_core_components as dcc
import plotly.offline as py
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import plotly.tools as tls
import plotly.plotly as py
from plotly.widgets import GraphWidget
from datetime import datetime, timedelta
from pandas.api.types import is_string_dtype
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets


<IPython.core.display.Javascript object>

In [2]:
import plotly
plotly.offline.init_notebook_mode(connected=True)

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

In [3]:
app = dash.Dash('Dash for EdX Analysis')

In [4]:
pc = pd.read_csv('sample_data/data_person_course.csv')
pcd = pd.read_csv('sample_data/data_person_course_day.csv')
sasby = pd.read_csv('sample_data/data_show_ans_stat_by_user.csv')
cg = pd.read_csv('sample_data/data_chapter_grades.csv')
pg = pd.read_csv('sample_data/data_problem_grades.csv')
tot = pd.read_csv('sample_data/data_time_on_task.csv')
tott = pd.read_csv('sample_data/data_time_on_task_total.csv')

In [5]:
def scatter_function(xvalue, yvalue, textvalue, sizevalue, sizerefvalue, colorvalue, colorbartitle, namevalue, title, xlabel, labeltype, ylabel):
    
    COLORSCALE = [ [0, "rgb(244,236,21)"], [0.3, "rgb(249,210,41)"], [0.4, "rgb(134,191,118)"], 
                [0.5, "rgb(37,180,167)"], [0.65, "rgb(17,123,215)"], [1, "rgb(54,50,153)"] ]

    trace0 = go.Scatter(x=xvalue,
                        y=yvalue,
                        text=textvalue,
                        mode = 'markers',
                        marker=dict(
                            size=sizevalue,
                            sizeref = sizerefvalue,
                            sizemode = 'diameter',
                            color=colorvalue,
                            colorscale=COLORSCALE,
                            colorbar = dict( title = colorbartitle ),
                            line = dict( color = '#444' ),
                            reversescale = True,   
                            opacity = 0.7),
                        name= namevalue )

    data = [trace0]

    layout = go.Layout(title=title,
                   xaxis=dict(title=xlabel,type=labeltype),
                   yaxis=dict(title=ylabel),
                   hovermode = 'closest')

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)  
  

In [6]:
scatter_function((pc.nseek_video+pc.npause_video), (pc.nvideos_total_watched * 100), pc.user_id, pc.sum_dt, 10000, pc.nvideo, "nvideo", 'video stats', 'video stats', 'video events', 'log', 'nvideos_total_watched (percentage)')

In [7]:
scatter_function(sasby.n_show_answer_problem_seen, sasby.n_problems_seen, sasby.user_id, sasby.pct_show_answer_problem_seen, 2, sasby.n_attempted, 'Percent', 'problems_seen', 'show_answer stats', 'problems', 'linear', 'show_answer')

In [8]:
scatter_function((pc.nseek_video+pc.npause_video), (pc.nshow_answer), pc.user_id, pc.sum_dt, 10000, pc.nvideo, "nvideo", 'video stats', 'video stats', 'video events', 'log', 'nshow_answer')

## Removing Outliers

In [9]:
#remove two outliers

mo = pc[pc.user_id != 534636]
mo = mo[mo.user_id != 534814]

scatter_function((mo.nseek_video+mo.npause_video), (mo.nshow_answer), mo.user_id, mo.sum_dt, 10000, mo.nvideo, "nvideo", 'video stats', 'video stats', 'video events', 'log', 'nshow_answer')


In [10]:
scatter_function((mo.nseek_video+pc.npause_video), (mo.nvideos_total_watched * 100), mo.user_id, mo.sum_dt, 10000, mo.nvideo, "nvideo", 'video stats', 'video stats', 'video events', 'log', 'nvideos_total_watched (percentage)') 

In [11]:
def display(columns, df, title, xlabel, ylabel):
    traces = []
    for col in columns:
        #print df[col]
        trace = go.Scatter(x=df.index,y=df[col])
        traces.append(trace)
    
    layout = go.Layout(title=title,
                       hovermode = 'closest',
                       xaxis=dict(title=xlabel),
                       yaxis=dict(title=ylabel)
                      )
    fig = go.Figure(data=traces, layout=layout)
    iplot(fig)

In [12]:
total = pcd.groupby('date').sum()
display(['nproblems_attempted','nproblems_answered'],total,'nproblems_attempted vs. nproblems_answered', 'date', 'total')

In [13]:
total = pcd.groupby('date').sum()
display(['nvideo','nproblems_attempted'],total,'nvideo vs. nproblems_attempted', 'total', 'date')

In [14]:
total = pcd.groupby('date').mean()

def update(uname):

    user = pcd[pcd.username==uname]
    user_total = user.groupby('date').sum().reset_index()
    user_total.loc[:,('date')] = pd.to_datetime(user_total['date'])

    date_init = datetime(2016,8,8)
    date_last =  datetime(2016,12,29)

    user_days = user_total[(user_total.date >= date_init) & (user_total.date < date_last)]

    trace0 = go.Scatter(x=user_days.date,
                    y=(user_days.nevents),
                    text=user_days.date,
                    name='{}'.format(uname))
    trace1 = go.Scatter(x=user_days.date,
                    y=total.nevents,
                    text=user_days.date,
                    name='average')
    

    data = [trace0,trace1]
    layout = dict(
        title='Time series for user: {}'.format(uname),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                dict(count=3,
                     label='3d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=21,
                    label='3w',
                    step='day',
                    stepmode='backward'),
                dict(count=42,
                    label='6w',
                    step='day',
                    stepmode='backward'),
                dict(step='all')
                ])
            ),
            rangeslider=dict(),
        type='date'
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

uname = widgets.Text(
    value='cqv71yy',
    placeholder='',
    description='Username:',
    disabled=False
)
    

widgets.interactive(update,uname=uname)

interactive(children=(Text(value='cqv71yy', description='Username:', placeholder=''), Output()), _dom_classes=('widget-interact',))

In [15]:
def update(uname):

    user = pcd[pcd.username==uname]
    user_total = user.groupby('date').sum().reset_index()
    user_total.loc[:,('date')] = pd.to_datetime(user_total['date'])

    date_init = datetime(2016,8,8)
    date_last =  datetime(2016,12,29)

    user_days = user_total[(user_total.date >= date_init) & (user_total.date < date_last)]
    
    trace0 = go.Scatter(x=user_days.date,
                    y=user_days.nproblems_answered,
                    text=user_days.date,
                    marker=dict(
                        color='rgb(31, 119, 180)'),
                    name='answered')
    trace1 = go.Scatter(x=user_days.date,
                    y=total.nproblems_answered,
                    text=user_days.date,
                    marker=dict(
                        color='rgba(31, 119, 180, 1)'),
                    line = dict(
                        width = 2,
                        dash = 'dot'),
                    name='answered average')
    trace2 = go.Scatter(x=user_days.date,
                    y=user_days.nproblems_attempted,
                    text=user_days.date,
                    marker=dict(
                        color='rgb(255, 127, 14)'),
                    name='attempted')
    
    trace3 = go.Scatter(x=user_days.date,
                    y=total.nproblems_attempted,
                    text=user_days.date,
                    marker=dict(
                        color='rgba(255, 127, 14, 1)'),
                    line = dict(
                        width = 2,
                        dash = 'dot'),
                    name='attempted average')
    trace4 = go.Scatter(x=user_days.date,
                    y=user_days.nshow_answer,
                    text=user_days.date,
                    marker=dict(
                        color='rgb(44, 160, 44)'),
                    name='nshow_answer')
    trace5 = go.Scatter(x=user_days.date,
                    y=total.nshow_answer,
                    text=user_days.date,
                    marker=dict(
                        color='rgba(44, 160, 44, 1)'),
                    line = dict(
                        width = 2,
                        dash = 'dot'),
                    name='nshow_answer average')

    data = [trace0,trace1,trace2,trace3,trace4,trace5]

    layout = dict(
        title='Time series for user: {}'.format(uname),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                dict(count=3,
                     label='3d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=21,
                    label='3w',
                    step='day',
                    stepmode='backward'),
                dict(count=42,
                    label='6w',
                    step='day',
                    stepmode='backward'),
                dict(step='all')
                ])
            ),
            rangeslider=dict(),
        type='date'
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

uname = widgets.Text(
    value='cqv71yy',
    placeholder='',
    description='Username:',
    disabled=False
)
    

widgets.interactive(update,uname=uname)

interactive(children=(Text(value='cqv71yy', description='Username:', placeholder=''), Output()), _dom_classes=('widget-interact',))

In [16]:
def update_grades(uid):

    user = cg[cg.user_id==int(uid)]
    
    user_total = user.groupby('due_date_max').sum().reset_index()
    user_total.loc[:,('due_date_max')] = pd.to_datetime(user_total['due_date_max'])
    
    date_init = datetime(2016,8,8)
    date_last =  datetime(2016,12,29)

    user_days = user_total[(user_total.due_date_max >= date_init) & (user_total.due_date_max < date_last)]

    trace0 = go.Scatter(x=user_days.due_date_max,
                   y=((user_days.chgrade / user_days.user_chapter_max_grade) * 100),
                   text=user_days.due_date_max,
                   name='user {}'.format(uid))
    trace1 = go.Scatter(x=user_days.due_date_max,
                   y=((user_days.median_grade / user_days.chmax) * 100),
                   text=user_days.due_date_max,
                   name='class average')

    data = [trace0,trace1]
    
    layout = dict(
        title='chgrades for user {}'.format(uid),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                dict(count=3,
                     label='3d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=21,
                    label='3w',
                    step='day',
                    stepmode='backward'),
                dict(count=42,
                    label='6w',
                    step='day',
                    stepmode='backward'),
                dict(step='all')
                ])
            ),
            rangeslider=dict(),
        type='date'
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

uid = widgets.Text(
    value='534786',
    placeholder='',
    description='User ID:',
    disabled=False
)
    
widgets.interactive(update_grades,uid=uid)

interactive(children=(Text(value='534786', description='User ID:', placeholder=''), Output()), _dom_classes=('widget-interact',))

In [17]:
def update_grades(uid):

    user = cg[cg.user_id==int(uid)]
    
    user_total = user.groupby('due_date_max').sum().reset_index()
    user_total.loc[:,('due_date_max')] = pd.to_datetime(user_total['due_date_max'])
    
    date_init = datetime(2016,8,8)
    date_last =  datetime(2016,12,29)

    user_days = user_total[(user_total.due_date_max >= date_init) & (user_total.due_date_max < date_last)]

    trace0 = go.Bar(x=user_days.due_date_max,
                   y=((user_days.chgrade / user_days.user_chapter_max_grade) * 100),
                   text=user_days.due_date_max,
                   name='user {}'.format(uid))
    trace1 = go.Bar(x=user_days.due_date_max,
                   y=((user_days.median_grade / user_days.chmax) * 100),
                   text=user_days.due_date_max,
                   name='class average')

    data = [trace0,trace1]
    
    layout = dict(
        title='chgrades for user {}'.format(uid),
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                dict(count=3,
                     label='3d',
                     step='day',
                     stepmode='backward'),
                dict(count=7,
                     label='1w',
                     step='day',
                     stepmode='backward'),
                dict(count=21,
                    label='3w',
                    step='day',
                    stepmode='backward'),
                dict(count=42,
                    label='6w',
                    step='day',
                    stepmode='backward'),
                dict(step='all')
                ])
            ),
            rangeslider=dict(),
        type='date'
        )
    )

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

uid = widgets.Text(
    value='534786',
    placeholder='',
    description='User ID:',
    disabled=False
)
    
widgets.interactive(update_grades,uid=uid)

interactive(children=(Text(value='534786', description='User ID:', placeholder=''), Output()), _dom_classes=('widget-interact',))

In [18]:
trace0 = go.Scatter(x=sasby.n_problems_seen,
                   y=sasby.n_show_answer_problem_seen,
                   text=sasby.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=sasby.n_problems_seen,
                       sizeref = 25,
                       sizemode = 'diameter'),
                   name='problems_seen')

trace1 = go.Scatter(x=sasby.n_attempted,
                   y=sasby.n_show_answer_attempted,
                   text=sasby.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=sasby.n_attempted,
                       sizeref = 25,
                       sizemode = 'diameter'),
                   name='problems_attempted')

trace2 = go.Scatter(x=sasby.n_not_attempted,
                   y=sasby.n_show_answer_not_attempted,
                   text=sasby.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=sasby.n_not_attempted,
                       sizeref = 25,
                       sizemode = 'diameter'),
                   name='problems_not_attempted')

trace3 = go.Scatter(x=sasby.n_perfect,
                   y=sasby.n_show_answer_perfect,
                   text=sasby.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=sasby.n_perfect,
                       sizeref = 25,
                       sizemode = 'diameter'),
                   name='problems_perfect')

trace4 = go.Scatter(x=sasby.n_partial,
                   y=sasby.n_show_answer_partial,
                   text=sasby.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=sasby.n_partial,
                       sizeref = 25,
                       sizemode = 'diameter'),
                   name='problems_partial')

data = [trace0,trace1,trace2,trace3,trace4]

layout = go.Layout(title="show_answer stats",
                   xaxis=dict(title='problems',type='linear'),
                   yaxis=dict(title='show_answer'),
                   hovermode = 'closest')

fig = go.Figure(data=data, layout=layout)
iplot(fig)  



In [19]:
def update(uid):
    user = sasby[sasby.user_id==int(uid)]
    current_user = user.groupby('user_id').sum().reset_index()

    
    trace0 = go.Scatter(x=current_user.n_problems_seen,
                   y=current_user.n_show_answer_problem_seen,
                   text=current_user.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=current_user.n_problems_seen,
                       sizeref = 2,
                       sizemode = 'diameter'),
                   name='problems_seen')
    
    trace1 = go.Scatter(x=current_user.n_attempted,
                   y=current_user.n_show_answer_attempted,
                   text=current_user.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=current_user.n_attempted,
                       sizeref = 2,
                       sizemode = 'diameter'),
                   name='problems_attempted')
    
    trace2 = go.Scatter(x=current_user.n_not_attempted,
                   y=current_user.n_show_answer_not_attempted,
                   text=current_user.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=current_user.n_not_attempted,
                       sizeref = 2,
                       sizemode = 'diameter'),
                   name='problems_not_attempted')
    
    trace3 = go.Scatter(x=current_user.n_perfect,
                   y=current_user.n_show_answer_perfect,
                   text=current_user.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=current_user.n_perfect,
                       sizeref = 2,
                       sizemode = 'diameter'),
                   name='problems_perfect')
    
    trace4 = go.Scatter(x=current_user.n_partial,
                   y=current_user.n_show_answer_partial,
                   text=current_user.user_id,
                   mode = 'markers',
                   marker=dict(
                       size=current_user.n_partial,
                       sizeref = 2,
                       sizemode = 'diameter'),
                   name='problems_partial')
    
    data = [trace0,trace1,trace2,trace3,trace4]

    layout = dict(
        title='nshow_answer stats for: {}'.format(uid),
        xaxis=dict(title='problems',type='linear'),
        yaxis=dict(title='show_answer'),
                   hovermode = 'closest')

    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

    
    

uid = widgets.Text(
    value='534715',
    placeholder='',
    description='User ID:',
    disabled=False
)
    

widgets.interactive(update,uid=uid)

interactive(children=(Text(value='534715', description='User ID:', placeholder=''), Output()), _dom_classes=('widget-interact',))

In [20]:
def update_donut(uid):
    user = sasby[sasby.user_id==int(uid)]
    current_user = user.groupby('user_id').sum().reset_index()
    
    attempted = int(current_user.n_attempted)
    
    hoverinfo = 'label+percent+name'
    hole = 0.4
        
    incorrect = int(current_user.n_attempted) - int(current_user.n_partial)
    partially_correct = int(current_user.n_partial) - int(current_user.n_perfect)
    correct = int(current_user.n_perfect)
    
    labels1 = ['Incorrect','Partially Correct','Correct']
    values1 = [incorrect, partially_correct, correct]
    
    domain1 = dict(x=[0,0.48],
                   y=[0.51,1])
    
    marker1 = dict(colors=['rgba(163, 203, 56, 1)','rgba(18, 137, 167, 1)','rgba(87, 88, 187, 1)'])
    
    name1 = 'Problems'
    
    trace1 = go.Pie(labels=labels1,
                    values=values1,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain1,
                    marker=marker1,
                    name=name1)
    
    nsa_incorrect = (int(current_user.n_show_answer_attempted) - int(current_user.n_show_answer_partial))
    incorrect_not_nsa = incorrect - nsa_incorrect
    
    
    labels2 = ['show answer','no show answer']
    values2 = [nsa_incorrect,incorrect_not_nsa]
    
    domain2 = dict(x=[0.52,1],
                   y=[0.51,1])
    
    marker2 = dict(colors=['rgba(163, 203, 56, 1)','rgba(163, 203, 56, 0.5)'])
    
    name2 = 'Incorrect Answers'
    
    trace2 = go.Pie(labels=labels2,
                    values=values2,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain2,
                    marker=marker2,
                    name=name2)
    
    nsa_partially_correct = (int(current_user.n_show_answer_partial) - int(current_user.n_show_answer_perfect))
    partially_correct_not_nsa = partially_correct - nsa_partially_correct
    
    
    labels3 = ['show answer','no show answer']
    values3 = [nsa_partially_correct,partially_correct_not_nsa]
    
    domain3 = dict(x=[0,0.48],
                   y=[0,0.49])
    
    marker3 = dict(colors=['rgba(18, 137, 167, 1)','rgba(18, 137, 167, 0.5)']) 
    
    name3 = 'Partially Correct Answers'
    
    trace3 = go.Pie(labels=labels3,
                    values=values3,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain3,
                    marker=marker3,
                    name=name3)
    
    nsa_correct = int(current_user.n_show_answer_perfect)
    correct_not_nsa = correct - nsa_correct
    
    labels4 = ['show answer','no show answer']
    values4 = [nsa_correct,correct_not_nsa]
    
    domain4 = dict(x=[0.52,1],
                   y=[0,0.49])
    
    marker4 = dict(colors=['rgba(87, 88, 187, 1)','rgba(87, 88, 187, 0.5)'])
    
    name4 = 'Correct Answers'
    
    trace4 = go.Pie(labels=labels4,
                    values=values4,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain4,
                    marker=marker4,
                    name=name4)
    
    layout = go.Layout(title="show_answer stats",
                       showlegend=False,
                       hovermode = 'closest')
    
    data = [trace1,trace2,trace3,trace4]
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)


uid = widgets.Text(
    value='534714',
    placeholder='',
    description='User ID:',
    disabled=False
)
    

widgets.interactive(update_donut,uid=uid)


interactive(children=(Text(value='534714', description='User ID:', placeholder=''), Output()), _dom_classes=('widget-interact',))

In [21]:
def average_donut():
    
    avg_user = sasby.mean()
    
    attempted = int(avg_user.n_attempted)
    
    hoverinfo = 'label+percent+name'
    hole = 0.4
        
    incorrect = int(avg_user.n_attempted) - int(avg_user.n_partial)
    partially_correct = int(avg_user.n_partial) - int(avg_user.n_perfect)
    correct = int(avg_user.n_perfect)
    
    labels1 = ['Incorrect','Partially Correct','Correct']
    values1 = [incorrect, partially_correct, correct]
    
    domain1 = dict(x=[0,0.48],
                   y=[0.51,1])
    
    marker1 = dict(colors=['rgba(163, 203, 56, 1)','rgba(18, 137, 167, 1)','rgba(87, 88, 187, 1)'])
    
    name1 = 'Problems'
    
    trace1 = go.Pie(labels=labels1,
                    values=values1,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain1,
                    marker=marker1,
                    name=name1)
    
    nsa_incorrect = (int(avg_user.n_show_answer_attempted) - int(avg_user.n_show_answer_partial))
    incorrect_not_nsa = incorrect - nsa_incorrect
    
    
    labels2 = ['show answer','no show answer']
    values2 = [nsa_incorrect,incorrect_not_nsa]
    
    domain2 = dict(x=[0.52,1],
                   y=[0.51,1])
    
    marker2 = dict(colors=['rgba(163, 203, 56, 1)','rgba(163, 203, 56, 0.5)'])
    
    name2 = 'Incorrect Answers'
    
    trace2 = go.Pie(labels=labels2,
                    values=values2,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain2,
                    marker=marker2,
                    name=name2)
    
    nsa_partially_correct = (int(avg_user.n_show_answer_partial) - int(avg_user.n_show_answer_perfect))
    partially_correct_not_nsa = partially_correct - nsa_partially_correct
    
    
    labels3 = ['show answer','no show answer']
    values3 = [nsa_partially_correct,partially_correct_not_nsa]
    
    domain3 = dict(x=[0,0.48],
                   y=[0,0.49])
    
    marker3 = dict(colors=['rgba(18, 137, 167, 1)','rgba(18, 137, 167, 0.5)']) 
    
    name3 = 'Partially Correct Answers'
    
    trace3 = go.Pie(labels=labels3,
                    values=values3,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain3,
                    marker=marker3,
                    name=name3)
    
    nsa_correct = int(avg_user.n_show_answer_perfect)
    correct_not_nsa = correct - nsa_correct
    
    labels4 = ['show answer','no show answer']
    values4 = [nsa_correct,correct_not_nsa]
    
    domain4 = dict(x=[0.52,1],
                   y=[0,0.49])
    
    marker4 = dict(colors=['rgba(87, 88, 187, 1)','rgba(87, 88, 187, 0.5)'])
    
    name4 = 'Correct Answers'
    
    trace4 = go.Pie(labels=labels4,
                    values=values4,
                    hoverinfo=hoverinfo,
                    hole=hole,
                    domain=domain4,
                    marker=marker4,
                    name=name4)
    
    layout = go.Layout(title="show_answer stats for average user",
                       showlegend=False,
                       hovermode = 'closest')
    
    data = [trace1,trace2,trace3,trace4]
    
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

average_donut()

In [24]:
BACKGROUND = 'rgb(230, 230, 230)'

COLORSCALE = [ [0, "rgb(244,236,21)"], [0.3, "rgb(249,210,41)"], [0.4, "rgb(134,191,118)"], 
                [0.5, "rgb(37,180,167)"], [0.65, "rgb(17,123,215)"], [1, "rgb(54,50,153)"] ]

def show_answer_incorrect_v_correct( 
        x = (sasby['n_show_answer_perfect']), 
        y = (sasby['n_show_answer_partial'] - sasby['n_show_answer_perfect']), 
        #size = sasby['n_perfect'],
        color = (sasby['n_partial'] - sasby['n_perfect']),    
        xlabel = 'video events',
        ylabel = 'nshow_answer',
        title='All Users',
        plot_type = 'scatter',
        markers = [] ):

    def axis_template_2d(title):
        return dict(
            backgroundcolor = BACKGROUND,
            gridcolor = 'rgb(255, 255, 255)',
            title = title,
            zerolinecolor = 'rgb(255, 255, 255)',
            color = '#444'
        )        
    
    def blackout_axis( axis ):
        axis['showgrid'] = False
        axis['zeroline'] = False
        axis['color']  = 'white'
        return axis
    
    data = [ dict(
        x = x,
        y = y,
        mode = 'markers',
        marker = dict( 
                colorscale = COLORSCALE,
                colorbar = dict( title = "incorrect problems" ),
                line = dict( color = '#444' ),
                reversescale = True,   
                #sizeref = 10,
                sizemode = 'diameter',
                opacity = 0.7,                
                size = 12,    
                color = color,
            ),
        text = pc['user_id'],
        type = plot_type,      
    ) ]
    
    layout = dict(
        title = 'All Users',
        xaxis = dict(
            title='video events',
            type='log'),
        yaxis = dict(
            title='nshow_answer_incorrect'),
        hovermode = 'closest',
        margin = dict( r=0, t=0, l=0, b=0 ),
        showlegend = False,
        scene = dict(
            xaxis = axis_template_2d( xlabel ),
            yaxis = axis_template_2d( ylabel ),
            camera = dict(
                up=dict(x=0, y=0),
                center=dict(x=0, y=0),
                eye=dict(x=0.08, y=2.2)
            )            
        )
    )
    
        
    if plot_type == 'histogram2d':
        # Scatter plot overlay on 2d Histogram
        data[0]['type'] = 'scatter'
        data.append( dict(
                x = x,
                y = y,
                type = 'histogram2d',
                colorscale = 'Greys',
                showscale = False
            ) )
        layout['plot_bgcolor'] = 'black'
        layout['paper_bgcolor'] = 'black'
        layout['xaxis'] = blackout_axis(layout['xaxis'])
        layout['yaxis'] = blackout_axis(layout['yaxis'])
        layout['font']['color'] = 'white'
        layout['title'] = 'All Users'
    
    #if len(markers) > 0:
        #data = data + add_markers( data, markers, plot_type = plot_type )
    layout = go.Layout(title = 'All Users',
                       xaxis=dict(
                           title='nshow_answer_perfect',
                           type = 'linear'),
                       yaxis=dict(
                           title='nshow_answer_incorrect'),
                      hovermode = 'closest')
    
    return dict( data=data, layout=layout )

In [25]:
py.iplot( show_answer_incorrect_v_correct() )

In [26]:
def nproblem_v_prog_check( 
        x = (pc['nproblem_check']), 
        y = (pc['nprogcheck']), 
        #size = sasby['n_perfect'],
        color = (pc['nevents']),    
        xlabel = 'nproblem_check',
        ylabel = 'nprogcheck',
        title='All Users',
        plot_type = 'scatter',
        markers = [] ):

    def axis_template_2d(title):
        return dict(
            backgroundcolor = BACKGROUND,
            gridcolor = 'rgb(255, 255, 255)',
            title = title,
            zerolinecolor = 'rgb(255, 255, 255)',
            color = '#444'
        )        
    
    def blackout_axis( axis ):
        axis['showgrid'] = False
        axis['zeroline'] = False
        axis['color']  = 'white'
        return axis
    
    data = [ dict(
        x = x,
        y = y,
        mode = 'markers',
        marker = dict( 
                colorscale = COLORSCALE,
                colorbar = dict( title = "nevents" ),
                line = dict( color = '#444' ),
                reversescale = True,   
                #sizeref = 10,
                sizemode = 'diameter',
                opacity = 0.7,                
                size = 12,    
                color = color,
            ),
        text = pc['user_id'],
        type = plot_type,      
    ) ]
    
    layout = dict(
        title = 'All Users',
        xaxis = dict(
            title='nproblem_check',
            type='log'),
        yaxis = dict(
            title='nprogcheck'),
        hovermode = 'closest',
        margin = dict( r=0, t=0, l=0, b=0 ),
        showlegend = False,
        scene = dict(
            xaxis = axis_template_2d( xlabel ),
            yaxis = axis_template_2d( ylabel ),
            camera = dict(
                up=dict(x=0, y=0),
                center=dict(x=0, y=0),
                eye=dict(x=0.08, y=2.2)
            )            
        )
    )
    
        
    if plot_type == 'histogram2d':
        # Scatter plot overlay on 2d Histogram
        data[0]['type'] = 'scatter'
        data.append( dict(
                x = x,
                y = y,
                type = 'histogram2d',
                colorscale = 'Greys',
                showscale = False
            ) )
        layout['plot_bgcolor'] = 'black'
        layout['paper_bgcolor'] = 'black'
        layout['xaxis'] = blackout_axis(layout['xaxis'])
        layout['yaxis'] = blackout_axis(layout['yaxis'])
        layout['font']['color'] = 'white'
        layout['title'] = 'All Users'
    
    #if len(markers) > 0:
        #data = data + add_markers( data, markers, plot_type = plot_type )
    layout = go.Layout(title = 'All Users',
                       xaxis=dict(
                           title='nproblem_check',
                           type = 'linear'),
                       yaxis=dict(
                           title='nprogcheck'),
                      hovermode = 'closest')
    
    return dict( data=data, layout=layout )

In [27]:
py.iplot( nproblem_v_prog_check() )

In [28]:
def nperfect_v_nprogcheck( 
        x = (sasby['n_perfect']), 
        y = (pc['nprogcheck']), 
        #size = sasby['n_perfect'],
        color = (sasby['n_not_attempted']),    
        xlabel = 'n_perfect',
        ylabel = 'nprogcheck',
        title='All Users',
        plot_type = 'scatter',
        markers = [] ):

    def axis_template_2d(title):
        return dict(
            backgroundcolor = BACKGROUND,
            gridcolor = 'rgb(255, 255, 255)',
            title = title,
            zerolinecolor = 'rgb(255, 255, 255)',
            color = '#444'
        )        
    
    def blackout_axis( axis ):
        axis['showgrid'] = False
        axis['zeroline'] = False
        axis['color']  = 'white'
        return axis
    
    data = [ dict(
        x = x,
        y = y,
        mode = 'markers',
        marker = dict( 
                colorscale = COLORSCALE,
                colorbar = dict( title = "nevents" ),
                line = dict( color = '#444' ),
                reversescale = True,   
                #sizeref = 2,
                sizemode = 'diameter',
                opacity = 0.7,                
                size = 12,    
                color = color,
            ),
        text = pc['user_id'],
        type = plot_type,      
    ) ]
    
    layout = dict(
        title = 'All Users',
        xaxis = dict(
            title='n_perfect',
            type='log'),
        yaxis = dict(
            title='nprogcheck'),
        hovermode = 'closest',
        margin = dict( r=0, t=0, l=0, b=0 ),
        showlegend = False,
        scene = dict(
            xaxis = axis_template_2d( xlabel ),
            yaxis = axis_template_2d( ylabel ),
            camera = dict(
                up=dict(x=0, y=0),
                center=dict(x=0, y=0),
                eye=dict(x=0.08, y=2.2)
            )            
        )
    )
    
        
    if plot_type == 'histogram2d':
        # Scatter plot overlay on 2d Histogram
        data[0]['type'] = 'scatter'
        data.append( dict(
                x = x,
                y = y,
                type = 'histogram2d',
                colorscale = 'Greys',
                showscale = False
            ) )
        layout['plot_bgcolor'] = 'black'
        layout['paper_bgcolor'] = 'black'
        layout['xaxis'] = blackout_axis(layout['xaxis'])
        layout['yaxis'] = blackout_axis(layout['yaxis'])
        layout['font']['color'] = 'white'
        layout['title'] = 'All Users'
    
    #if len(markers) > 0:
        #data = data + add_markers( data, markers, plot_type = plot_type )
    layout = go.Layout(title = 'All Users',
                       xaxis=dict(
                           title='n_perfect',
                           type = 'linear'),
                       yaxis=dict(
                           title='nprogcheck',
                           type = 'log'),
                      hovermode = 'closest')
    
    return dict( data=data, layout=layout )

In [29]:
py.iplot( nperfect_v_nprogcheck() )